# Basic homomorphic encryption
Let's start with some basic homomorphic encryption using exponentiation of a generator and modulo

The first step is to get a polynomial evaluated in the unencrypted domain, encrypt the result.
The second step is to evaluate it in the encrypted domain.
The third step is to verify whether they match, or not.

$$ p(x)= c0 + c1*x + c2*x^2 +c3*x^3 $$

The other way to represent this polynomial is by using co-factors, which uses the roots (solution) of the polynomial.

$$ p(x) = (x - r1) * (x - r2) * (x - r3) = 0 $$


In [183]:
from IPython.display import Markdown as md
import numpy as np
# Values for the user to input, if he wants to
generator = 7
field_prime = 241
#field_prime = 7919

# x^3 - 18 x^2 + 80 x - 96 = 0
# Co-factors (roots) for this polynomial are [2,4,12]

# Let's compute the coefficents from known roots and see what we get. 
# We'll use numpy's integrated function to do this
polynomial_roots = [10,3,5]
calculated_coefficients = np.polynomial.polynomial.polyfromroots(polynomial_roots)
#print(calculated_coefficients)
#p_polynomial_coefficients = [96,80,-18,1]
p_polynomial_coefficients = list(map(int,calculated_coefficients.astype(int))) # map back to a list of int
#print(p_polynomial_coefficients)
target_polynomial_coefficients = []

# change_variables = input("Do you want to change variables?")
# if change_variables == "yes":
#     generator = input("generator")

equation = "$$ p(x)= "

for index, coeff in enumerate(p_polynomial_coefficients):

    if index == 0:
        equation+= str(coeff)
    else:
        if coeff >= 0:
            equation+= (" + ")
        else:
            equation+= (" - ")
        equation+= str(abs(coeff))
        equation+= (" * x^{}".format(index))
               
equation+= (" $$")
md(equation)
#md("$$ p(x)= {} + {}*x + c2*x^2 $$".format(p_polynomial_coefficients[0],p_polynomial_coefficients[1]))

$$ p(x)= -150 + 95 * x^1 - 18 * x^2 + 1 * x^3 $$

In [184]:
# define the encryption function
# all operations defined on Galois field G(p), with p being a prime number
# encryption is done by exponentiation of a generator (a prime number too) to the power of the unencrypted value
# Now fortunately, the pow() function in Python has an embedded functionality to apply a modulo
#  and it even works on large numbers, by using reduction
def encrypt(unencrypted, gen=generator,modulo=field_prime):
    encrypted = pow(gen,unencrypted,modulo)
    #print("exponentiated to : ", encrypted)
    return encrypted
    
print("encrypt 5, with generator 7 and p 19: ",encrypt(5,gen=7,modulo=19))
print("encrypt 6, with generator 7 and p 19: ",encrypt(6,gen=7,modulo=19))
print("encrypt 334499, with generator 7 and p 19: ",encrypt(334499,gen=7,modulo=19))
print('let\'s what we get with negative number -5:',encrypt(-5,gen=7,modulo=19))
print('The negative number is the same thing as applying the exponentiation to the inverse multiplicative',encrypt(14,gen=7,modulo=19) )


encrypt 5, with generator 7 and p 19:  11
encrypt 6, with generator 7 and p 19:  1
encrypt 334499, with generator 7 and p 19:  11
let's what we get with negative number -5: 7
The negative number is the same thing as applying the exponentiation to the inverse multiplicative 11


## Now we see how we can do our first encryption of polynomial
The way to do it is first to compute the polynomial in the unencrypted domain for a given r(a scalar that is used to compute the value of the polynomial, e.g. p(r)), then encrypt this result using the method described above.
Then the prover computes it in the encrypted domain, returns it to the verifier.
Finally, the Verifier verifies the 2 values match.

In [185]:
# We have defined the polynomial earlier with its coefficients in p_polynomial_coefficients
# let's compute the polynomial p(r) value for a given r (not a root)
r=20
evaluated_poly = np.polynomial.polynomial.polyval(r, p_polynomial_coefficients)
print('the polynomial evaluated at r: ',evaluated_poly)
# Now let's encrypt this value and see what we get
encrypted_result = encrypt(int(evaluated_poly))
print('encrypted result: ',encrypted_result)

# The second part is to verify that we can get the same result when doing the computation in the encryted domain
# How we do this is by having the verifier (he chooses a secret value where to evaluate the polynomial) give the encrypted value
# of each x^i for the value r. In our case the prover needs the encrypted values for 11^3, 11^2, 11^1 and 11^0
# For now we just want to see it in action, we are not yet implementing the protocol for Zero Knowledge proof
encrypted_powers = []
for index, order in enumerate(p_polynomial_coefficients):
    encrypted_powers.append(encrypt(pow(r,index)))

# Now let's compute E (p(11)) = encrypted_powers[0]^coeff[0] * encrypted_powers[1]^coeff[1] * encrypted_powers[2]^coeff[2] * encrypted_powers[3]^coeff[3] 
# We apply modulo a few times along the way to reduce the chance of overflow
result = 1
for index,coeff in enumerate(p_polynomial_coefficients):
    result = (result * pow(encrypted_powers[index],int(coeff),field_prime)) % field_prime
print('the result is: ....',result)


the polynomial evaluated at r:  2550.0
encrypted result:  211
the result is: .... 211


In [186]:

# I think from here on out it would be good to have a verifier and a prover classes and clearly delineate the tasks they each
# must perform. As the functionality will evolve as we make the protocol more complex, the classes will be named 
# using the chapter number defining the protocol

# We will need random values from here on out, let's use the secrets module for it
import secrets

# There are some limitations with using the numpy package to evaluate the polynomial when using large numbers
# Because of this we will use a [package](https://mhostetter.github.io/galois/latest/tutorials/intro-to-prime-fields/#prime-field) that can do polynomial evaluation in a Galois Field
import galois

# A few things are different from the example given in the preceding section
# The verifier knows a polynomial t(x), called the target polynomial
# The prover claims he knows a polynomial of order n p(x) that is divisible by t(x) (thus has t(x)'s roots as co-factors)
# The prover wants to keep p(x) secret, but wants to prove he knows it
# Thus the prover computes p(x)/t(x) = h(x), and will provide the value of p(s) and h(s) to the verifier
# The prover uses s that is provided by the verifier, and the computation occurs in the encrypted domain
# As is mentioned in the pdf document, we cannot exponentiate in the encrypted domain
# That is why the verifier must provide the encrypted values of s for all orders of the polynomial
# That is E(s^0), E(s^1), ... , E(s^n), where n is the polynomial order
# Finally once the prover returns p(s) and h(s) to the verifier, the latter computes p(s)/t(s) and if it 
# matches p(s), then he knows it's correct (very bad protocol for now, but it'll get more secure)

# Because we are going to implement multiple version of the protocol, I'll use the protocol module
# There's a good explanation how it works at: https://idego-group.com/blog/2023/02/21/we-need-to-talk-about-protocols-in-python/
from typing import Protocol

class Verifier(Protocol):
    def __init__(self, target_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__()
        self.encrypted_powers = []
        self.field_prime = field_prime
        self.generator = generator
        self.target_polynomial_coefficients = target_polynomial_coefficients
        self.target_desc_coeffs = list(reversed(target_polynomial_coefficients))
        self.s : int
    
    def calculate_encrypted_powers(self):
        ...
    
    def evaluate_unencrypted_target_polynomial(self):
        ...
    
    def set_secret(self, s:int):
        ...
    

class Verifier_334(Verifier):
# The verifier's tasks are as follows:
# (a) sample a random value s, i.e., secret
# (b) calculate encryptions of s for all powers i in 0, 1, ..., d, i.e.: E(si) = gsi
# (c) evaluate unencrypted target polynomial with s: t(s)
# (d) encrypted powers of s are provided to the prover: E(s0), E(s1), ..., E(sd)

    def __init__(self, target_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__(target_polynomial_coefficients, field_prime, generator)
        self.s = secrets.randbelow(self.field_prime) # (a)
        print(f'secret = {self.s}')
    
    def calculate_encrypted_powers(self):
        # (b) & (d)
        for index, order in enumerate(self.target_polynomial_coefficients):
            self.encrypted_powers.append(encrypt(pow(self.s,index,field_prime)))

    def evaluate_unencrypted_target_polynomial(self):
        GF = galois.GF(self.field_prime) # create a Finite Field
        print(galois.primitive_root(self.field_prime))
        target_poly = galois.Poly(self.target_desc_coeffs, field=GF)
        evaluated_poly = target_poly(self.s)
        print(f'the polynomial evaluated at s = {self.s} is {evaluated_poly} ')
        # Now let's encrypt this value and see what we get
        self.encrypted_result = encrypt(int(evaluated_poly))
        print('encrypted result: ',self.encrypted_result)  

    def set_secret(self, s:int):
        self.s = s

# let's exercise this new class
# target poly is (x-1)(x-2), which is equivalent to 2 -3x +x^2
myVerifier =  Verifier_334([2,-3,1])
print(myVerifier.target_polynomial_coefficients)
myVerifier.evaluate_unencrypted_target_polynomial()
myVerifier.calculate_encrypted_powers()

# just to make sure everything works fine, let's test this implementation with the same values we used in the 
# preceding block
myValidatedVerifier = Verifier_334(p_polynomial_coefficients)
myValidatedVerifier.set_secret(20)
print(myValidatedVerifier.target_polynomial_coefficients)
myValidatedVerifier.evaluate_unencrypted_target_polynomial()

# it is not working, the reason is that any value used in the computation of the GF must be in the finite field.
# so what we see is that the evaluation of the polynomial in the first implementation returns 2550 for r=20
# With the GF, the evaluated value is 140, which is 2550 mod 241! For this test we have been using 241 as the field prime
# How can we explain that the prover's result matches the verifier's then? 
# it seems to me that there is a good chance that is because the power of r values are outside of the field limit
# So, if we take a field prime that is big enough, the results will match. Let's see if this works: new field prime = 7919
# Result is that it works indeed! But I think the field prime should be 

secret = 234
[2, -3, 1]
7


AssertionError: <code object __call__ at 0x134fe4e30, file "/Users/Jean-Marc/Documents/ZKsnark-notebook/.venv/lib/python3.12/site-packages/galois/_polys/_dense.py", line 409> != <code object implementation at 0x1101109e0, file "/Users/Jean-Marc/Documents/ZKsnark-notebook/.venv/lib/python3.12/site-packages/galois/_polys/_dense.py", line 432>

In [ ]:
evaluated_poly = np.polynomial.polynomial.polyval(r, p_polynomial_coefficients)
print('the polynomial evaluated at r: ',evaluated_poly)
# Now let's encrypt this value and see what we get
encrypted_result = encrypt(int(evaluated_poly))
print('encrypted result: ',encrypted_result)

# The second part is to verify that we can get the same result when doing the computation in the encryted domain
# How we do this is by having the verifier (he chooses a secret value where to evaluate the polynomial) give the encrypted value
# of each x^i for the value r. In our case the prover needs the encrypted values for 11^3, 11^2, 11^1 and 11^0
# For now we just want to see it in action, we are not yet implementing the protocol for Zero Knowledge proof
encrypted_powers = []
for index, order in enumerate(p_polynomial_coefficients):
    encrypted_powers.append(encrypt(pow(r,index)))

# Now let's compute E (p(11)) = encrypted_powers[0]^coeff[0] * encrypted_powers[1]^coeff[1] * encrypted_powers[2]^coeff[2] * encrypted_powers[3]^coeff[3] 
# We apply modulo a few times along the way to reduce the chance of overflow
result = 1
for index,coeff in enumerate(p_polynomial_coefficients):
    result = (result * pow(encrypted_powers[index],int(coeff),field_prime)) % field_prime
print('the result is: ....',result)

# I think from here on out it would be good to have a verifier functin and a prover function and clearly delineates the tasks they each
# must perform. As the functionality will evolve as we make the protocol more complex, the functions will be named 
# using the chapter number defining the protocol

# We will need random values from here on out, let's use the secrets module for it
import secrets

def verifier_334():
# The verifier's tasks are as follows:
# (a) sample a random value s, i.e., secret
# (b) calculate encryptions of s for all powers i in 0, 1, ..., d, i.e.: E(si) = gsi
# (c) evaluate unencrypted target polynomial with s: t(s)
# (d) encrypted powers of s are provided to the prover: E(s0), E(s1), ..., E(sd)

    s = secrets.randbelow(field_prime) # (a)

    encrypted_powers = [] # (b) & (d)
    for index, order in enumerate(p_polynomial_coefficients):
        encrypted_powers.append(encrypt(pow(s,index)))

     # let's compute the polynomial p(r) value with r = 11 (not a root)
    evaluated_poly2 = np.polynomial.polynomial.polyval(11, calculated_coefficients)
    print('the polynomial evaluated at 11: ',evaluated_poly2)
    # Now let's encrypt this value and see what we get
    encrypted_result2 = encrypt(int(evaluated_poly2))
    print('encrypted result: ',encrypted_result2)       


the polynomial evaluated at r:  2550.0
encrypted result:  211
the result is: .... 211
